# Hybrid Quantum Convolutional Neural Netwoks for Earth Observation Classification
![](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/4609443/9656571/9647979/sebas9-3134785-large.gif)


Please refer to the following articles to get more insight about this topic:
    - Sebastianelli, A., Zaidenberg, D. A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021). On circuit-based hybrid quantum neural networks for remote sensing imagery classification. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 15, 565-580.
    - Zaidenberg, D. A., Sebastianelli, A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021, July). Advantages and bottlenecks of quantum machine learning for remote sensing. In 2021 IEEE International Geoscience and Remote Sensing Symposium IGARSS (pp. 5680-5683). IEEE.
    - Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
    - Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.
    - https://qiskit.org/documentation/machine-learning/tutorials/index.html
    - https://pennylane.ai/qml/demos_qml.html


# Download the dataset

In this study, we address the challenge of land use and land cover classification using Sentinel-2 satellite images. The Sentinel-2 satellite images are openly and freely accessible provided in the Earth observation program Copernicus. We present a novel dataset based on Sentinel-2 satellite images covering 13 spectral bands and consisting out of 10 classes with in total 27,000 labeled and geo-referenced images. We provide benchmarks for this novel dataset with its spectral bands using state-of-the-art deep Convolutional Neural Network (CNNs). With the proposed novel dataset, we achieved an overall classification accuracy of 98.57%. The resulting classification system opens a gate towards a number of Earth observation applications. We demonstrate how this classification system can be used for detecting land use and land cover changes and how it can assist in improving geographical maps.

To download, click here: https://madm.dfki.de/files/sentinel/EuroSAT.zip
Unzip the downloaded package.



Get more information [here](https://github.com/phelber/EuroSAT).

![](https://github.com/phelber/EuroSAT/blob/master/eurosat_overview_small.jpg?raw=true)




For this tutorial we are using the EuroSAT RGB dataset, accessible via wget. The following cell takes care of downloading, unzipping and preparing the dataset. Add more here..... 270

# Network train and validation

In [1]:
# Suppressing warning
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from utils.DatasetHandler import DatasetHandler
from qc.QiskitCircuit import QiskitCircuit
from models.HybridNet import HybridNet
from models.PyTorchModel import PyTorchModel

# Configuration file, please read it carefully
from config import *

import torch.optim as optim
import torch.nn as nn
import torch
import os

# Allows to reload modified code without restarting the kernel
%load_ext autoreload
%autoreload 2

C:\Users\SeanWalsh\anaconda3\envs\conda_env\lib\site-packages\qiskit\validation\fields\custom.py:76: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.integer, numpy.float,
C:\Users\SeanWalsh\anaconda3\envs\conda_env\lib\site-packages\qiskit\providers\aer\pulse\qutip_lite\fastsparse.py:57: DeprecationWarning: Please use `upcast` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_index_dtype)
C:\Users\SeanWalsh\anaconda3\envs\conda_env\lib\site-packages\qiskit\providers\aer\pulse\qutip_lite\fastsparse.py:57: DeprecationWarning: Please use `isdense` from th

Load the dataset

In [2]:
################################ Initialize Dataset Handler ################################
################################ and print classes          ################################
print('Loading Dataset')
dh = DatasetHandler(DATASET_ROOT)

classes = []
for i, c in enumerate(dh.classes):
    cl = c.split(os.path.sep)[-1]
    classes.append(cl)
classes.sort()
print('[*] Classes: {}'.format(classes))

################################ Load image paths and labels ################################
imgs, labels = dh.load_paths_labels(DATASET_ROOT, classes=classes)
print('[*] Size: {}'.format(len(imgs)))

################################# Training-Validation Split #################################
tra_imgs, tra_lbls, val_imgs, val_lbls = dh.train_validation_split(imgs, labels, SPLIT_FACTOR)
print('[*] Training Size:   {}'.format(len(tra_imgs)))
print('[*] Validation Size: {}'.format(len(val_imgs)))

Loading Dataset
[*] Classes: ['2750']


KeyError: '2750'

Initialize hybrid model

In [3]:
print('Initialize Quantum Hybrid Neural Network')
circuit = QiskitCircuit()
network = HybridNet()

optimizer = optim.SGD(network.parameters(), lr=LEARNING_RATE, momentum = MOMENTUM)
criterion = nn.CrossEntropyLoss()

print('Printing Quantum Circuit')
print(circuit.circuit.draw(output='text', scale=1/NUM_LAYERS))

print('Printing Quantum Circuit Parameters')
print('[*] Number of Qubits:   {}'.format(NUM_QUBITS))
print('[*] Number of R Layers: {}'.format(NUM_LAYERS))
print('[*] Number of Outputs:  {}'.format(NUM_QC_OUTPUTS))
print('[*] Number of Shots:    {}'.format(NUM_SHOTS))

# This class wrap a PyTorch model. It is only needed to mask basic function, like model training.
model = PyTorchModel(network, criterion, optimizer)

Initialize Quantum Hybrid Neural Network
Printing Quantum Circuit
        ┌───┐ ░ ┌────────────┐┌───┐┌───┐┌───┐┌────────────┐                   »
   q_0: ┤ H ├─░─┤ RY(Theta0) ├┤ X ├┤ X ├┤ X ├┤ RY(Theta4) ├───────────────────»
        ├───┤ ░ ├────────────┤└─┬─┘└─┬─┘└─┬─┘└───┬───┬────┘┌───┐┌────────────┐»
   q_1: ┤ H ├─░─┤ RY(Theta1) ├──■────┼────┼──────┤ X ├─────┤ X ├┤ RY(Theta5) ├»
        ├───┤ ░ ├────────────┤       │    │      └─┬─┘     └─┬─┘└───┬───┬────┘»
   q_2: ┤ H ├─░─┤ RY(Theta2) ├───────■────┼────────■─────────┼──────┤ X ├─────»
        ├───┤ ░ ├────────────┤            │                  │      └─┬─┘     »
   q_3: ┤ H ├─░─┤ RY(Theta3) ├────────────■──────────────────■────────■───────»
        └───┘ ░ └────────────┘                                                »
meas_0: ══════════════════════════════════════════════════════════════════════»
                                                                              »
meas_1: ══════════════════════════════════════════════

Fit the model

In [4]:
tra_set = [tra_imgs, tra_lbls]
val_set = [val_imgs, val_lbls]
model.fit(EPOCHS, tra_set, val_set, classes, batch_size=BATCH_SIZE, es=None, tra_size = None, val_size = None)

NameError: name 'tra_imgs' is not defined

In [ ]:
model.curves()